In [1]:
import pandas as pd

df = pd.read_csv('test-data_encoded.csv')
df

parsed_smell  \
0    \n\t\t PropertyDeclaration\n\t\t\t GenericName...   
1    \n\t\t\t\t LocalDeclarationStatement\n\t\t\t\t...   
2                                \n\t ClassDeclaration   
3                                \n\t ClassDeclaration   
4    \n\t\t MethodDeclaration\n\t\t\t IdentifierNam...   
..                                                 ...   
310  \n\t\t PropertyDeclaration\n\t\t\t IdentifierN...   
311  \n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...   
312  \n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...   
313  \n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...   
314  \n\t\t MethodDeclaration\n\t\t\t PredefinedTyp...   

                              parsed_smell_identifiers    message  \
0    \n\t\t PropertyDeclaration\n\t\t\t GenericName...   Refactor   
1    \n\t\t\t\t LocalDeclarationStatement\n\t\t\t\t...   Refactor   
2                                \n\t ClassDeclaration  Deficient   
3                                \n\t ClassDeclaration  Deficient   
4    \n\t\t MethodDeclaration\n\t\t\t IdentifierNam...   Refactor   
..                                                 ...        ...   
310  \n\t\t PropertyDeclaration\n\t\t\t IdentifierN...   Refactor   
311  \n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...   Refactor   
312  \n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...   Refactor   
313  \n\t\t\t\t ExpressionStatement\n\t\t\t\t\t Inv...   Refactor   
314  \n\t\t MethodDeclaration\n\t\t\t PredefinedTyp...   Refactor   

     message_encoded  
0                  2  
1                  2  
2                  1  
3                  1  
4                  2  
..               ...  
310                2  
311                2  
312                2  
313                2  
314                2  

[315 rows x 4 columns]

In [2]:
import pandas as pd
import pickle

# Read the CSV file
df = pd.read_csv('test-data_encoded.csv')

# Extract the 'message_encoded' column
message_encoded = df['message_encoded']

# Save the message_encoded values to a pickle file
with open('messagelist.pkl', 'wb') as file:
    pickle.dump(message_encoded, file)

In [3]:
ast_list = df['parsed_smell'].tolist()
preprocessed_ast_list = [ast.strip() for ast in ast_list]

from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('bert-base-nli-mean-tokens')

ast_embeddings = model.encode(preprocessed_ast_list, convert_to_tensor=True)
save_path = 'parsed_smell_embeddings.pt'
torch.save(ast_embeddings, save_path)

loaded_ast_embeddings = torch.load(save_path)
loaded_ast_embeddings 

tensor([[-4.2081e-02,  7.4244e-01,  6.9006e-01,  ..., -5.6429e-01,
         -2.3851e-01,  1.3245e-01],
        [-4.8465e-01,  5.6752e-01,  7.7647e-01,  ..., -6.9026e-01,
         -3.3859e-01,  5.7747e-01],
        [-2.8201e-01, -3.5818e-01,  5.6016e-01,  ..., -3.8653e-01,
         -5.2939e-01, -1.2135e-01],
        ...,
        [-8.0790e-01,  6.3382e-01,  8.0822e-01,  ..., -8.2259e-01,
         -7.3171e-01,  8.1184e-01],
        [-6.8152e-01,  4.8239e-01,  6.2383e-01,  ..., -6.0730e-01,
         -6.4566e-01,  8.6671e-01],
        [-1.0384e-03, -4.0031e-01,  1.6763e+00,  ..., -5.9387e-01,
         -4.4608e-01,  1.3031e-01]])

In [4]:
ast_list = df['parsed_smell_identifiers'].tolist()
preprocessed_ast_list = [ast.strip() for ast in ast_list]

from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('bert-base-nli-mean-tokens')

ast_embeddings = model.encode(preprocessed_ast_list, convert_to_tensor=True)
save_path = 'parsed_smell_identifier_embeddings.pt'
torch.save(ast_embeddings, save_path)

loaded_ast_embeddings = torch.load(save_path)
loaded_ast_embeddings 

tensor([[-4.2081e-02,  7.4244e-01,  6.9006e-01,  ..., -5.6429e-01,
         -2.3851e-01,  1.3245e-01],
        [-4.6524e-01,  5.9399e-01,  8.6966e-01,  ..., -6.5632e-01,
         -3.2988e-01,  5.6402e-01],
        [-2.8201e-01, -3.5818e-01,  5.6016e-01,  ..., -3.8653e-01,
         -5.2939e-01, -1.2135e-01],
        ...,
        [-6.8476e-01,  8.8314e-01,  5.0515e-01,  ..., -7.4492e-01,
         -6.1222e-01,  7.3563e-01],
        [-5.0759e-01,  7.4027e-01,  8.5077e-01,  ..., -2.3173e-01,
         -5.6064e-01,  6.9322e-01],
        [-1.0384e-03, -4.0031e-01,  1.6763e+00,  ..., -5.9387e-01,
         -4.4608e-01,  1.3031e-01]])

In [9]:
num_embeddings = loaded_ast_embeddings.shape[0]
print(f"Number of saved embeddings: {num_embeddings}")

Number of saved embeddings: 315


In [12]:
import numpy as np

num_embeddings = len(ast_embeddings)

# Choose the index of the specific AST for comparison
ast_index = 22  # Replace with the desired AST index

similarity_scores = np.zeros(num_embeddings)

for i in range(num_embeddings):
    if i != ast_index:
        similarity_scores[i] = util.cos_sim(ast_embeddings[ast_index], ast_embeddings[i])

# Sort the similarity scores in descending order
sorted_indices = np.argsort(similarity_scores)[::-1]
sorted_scores = similarity_scores[sorted_indices]

# List the similarity scores for the specific AST in descending order
print(f"Ast {ast_index}:")
for i in range(num_embeddings - 1):
    idx = sorted_indices[i]
    score = sorted_scores[i]
    print(f"AST {idx}  {score:.2f}")

Ast 22:
AST 23  1.00
AST 24  1.00
AST 30  1.00
AST 21  1.00
AST 20  1.00
AST 31  1.00
AST 32  0.97
AST 85  0.96
AST 25  0.96
AST 26  0.96
AST 27  0.96
AST 313  0.95
AST 312  0.95
AST 311  0.95
AST 114  0.95
AST 115  0.95
AST 196  0.95
AST 67  0.95
AST 150  0.94
AST 92  0.94
AST 13  0.94
AST 185  0.94
AST 187  0.94
AST 181  0.94
AST 182  0.94
AST 183  0.94
AST 184  0.94
AST 186  0.94
AST 7  0.94
AST 180  0.94
AST 169  0.94
AST 170  0.94
AST 205  0.93
AST 59  0.93
AST 40  0.93
AST 15  0.93
AST 14  0.93
AST 34  0.93
AST 43  0.92
AST 88  0.92
AST 89  0.92
AST 91  0.92
AST 46  0.91
AST 206  0.91
AST 207  0.91
AST 38  0.91
AST 214  0.91
AST 212  0.91
AST 216  0.91
AST 219  0.91
AST 80  0.91
AST 195  0.91
AST 55  0.91
AST 133  0.91
AST 149  0.91
AST 147  0.91
AST 148  0.91
AST 190  0.91
AST 211  0.90
AST 213  0.90
AST 215  0.90
AST 194  0.90
AST 156  0.90
AST 151  0.90
AST 152  0.90
AST 153  0.90
AST 154  0.90
AST 155  0.90
AST 167  0.90
AST 158  0.90
AST 168  0.90
AST 159  0.90
AST 166  0.90

In [21]:
rows_to_print = [22,5 ]
columns_to_print = ['AST']
print(df.loc[rows_to_print, columns_to_print].to_string(index=False))

                                            AST
  \n\t\t\t PredefinedType\n\t\t\t ParameterList
\n\t\t\t\t Parameter\n\t\t\t\t\t PredefinedType
